```
curl 'https://investmap.pl/ajax2/map?category=1&ago=12m' \
  -H 'authority: investmap.pl' \
  -H 'accept: */*' \
  -H 'dnt: 1' \
  -H 'x-requested-with: XMLHttpRequest' \
  -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55' \
  -H 'referer: https://investmap.pl/mapa.html' \
  -H 'accept-language: pl,en;q=0.9,en-GB;q=0.8,en-US;q=0.7' \
  --compressed
```

```
curl 'https://investmap.pl/ajax2/map?category=1&estates=0&ago=' \
  -H 'authority: investmap.pl' \
  -H 'accept: */*' \
  -H 'dnt: 1' \
  -H 'x-requested-with: XMLHttpRequest' \
  -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55' \
  -H 'referer: https://investmap.pl/mapa.html' \
  -H 'accept-language: pl,en;q=0.9,en-GB;q=0.8,en-US;q=0.7' \
  --compressed
```

In [1]:
import requests
import json
import sqlite3
import os
import time

import logging
import logging.handlers

In [8]:
LOG_DIR = 'logs'

headers = {
    'authority': "investmap.pl",
    'accept': "*/*",
    "dnt": "1",
    # 'content-type': 'application/json',
    "x-requested-with": "XMLHttpRequest",
    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55",
    'referer': "https://investmap.pl/mapa.html",
    'accept-language': "pl,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
}

root = None

if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

In [15]:
conn = sqlite3.connect('rynek_pierwotny.db')
cur = conn.cursor()

In [16]:
#get the count of tables with the name
cur.execute('''SELECT count(name) FROM sqlite_master WHERE type='table' AND name='INVESTMAP' ''')

#if the count is 1, then table exists
if cur.fetchone()[0]==1 :
	print('Table exists.')
else:
    # Create table
    cur.execute('''CREATE TABLE "INVESTMAP" (
	"Id"	INTEGER NOT NULL,
	"Updated at"	TEXT,
	"Latitude"	REAL,
	"Longitude"	REAL,
	"Title"	TEXT,
	"Url"	TEXT,
	PRIMARY KEY("Id")
);''')
    conn.commit()

In [26]:
def get_all_fetched_ids():
    results_count = cur.execute("select Id from INVESTMAP")
    results = results_count.fetchall()
    return [r[0] for r in results]

def count_data_items(folder):
    return len([name for name in os.listdir(folder)])

def set_handler():
    global root
    index = count_data_items(LOG_DIR)
    handler = logging.handlers.WatchedFileHandler(
        os.environ.get("LOGFILE", f"logs/investmap_{index}.log"))
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    handler.setFormatter(formatter)
    root = logging.getLogger()
    root.setLevel(os.environ.get("LOGLEVEL", "DEBUG"))
    root.addHandler(handler)

def check_if_null(value):
    if value == None:
        return ':null'
    return value

def check_text(value):
    if value:
        value = value.replace('\\', '\\\\')
        value = value.replace('\'', '\'\'')
    return check_if_null(value)

In [37]:
def fetch_investmap():
    set_handler()
    root.info("Fetching offers from investmap")
    success = False

    while not success:
        try:
            url = "https://investmap.pl/ajax2/map?category=1&estates=0&ago=" # "https://investmap.pl/ajax2/map?category=1&ago="
            response = requests.get(url, headers=headers)
            json_response = json.loads(response.text)
            root.debug(f"Got {len(json_response)} response: {json_response}")

            success = True
        except Exception as e:
            root.error("Exception occurred", exc_info=True)
            time.sleep(30)
            root.info("Startating again")
            success = False

    for estate_json in json_response:
        # print(estate_json)
        root.debug(f"Adding to database: {estate_json}")
        id = estate_json['id']
        updated_at = estate_json['updated_at']
        latitude = estate_json['location']['lat'] if 'location' in estate_json else None
        longitude = estate_json['location']['lon'] if 'location' in estate_json else None
        title = check_text(estate_json['title'])
        url = check_text(estate_json['url'])

        insert_text = f"INSERT OR IGNORE INTO INVESTMAP VALUES ('{id}', '{updated_at}', {latitude}, {longitude}, '{title}', '{url}')"

        try:
            root.debug(insert_text)
            insert_result = cur.execute(insert_text, {'null':None})
            conn.commit()
        except Exception as error:
            root.error("Exception occurred", exc_info=True)

In [38]:
fetch_investmap()

## TRY CODE BELOW

In [17]:
url = "https://investmap.pl/ajax2/map?category=1&ago="
id = 14000
response = requests.get(url, headers=headers)
# response.content

In [12]:
json_response = json.loads(response.text)
json_response

[{'updated_at': '2016-11-04 13:10:00',
  'location': {'lon': 17.4609, 'lat': 50.8575},
  'id': 20,
  'title': 'Liberty Park',
  'url': '/inwestycja/liberty-park,20.html'},
 {'updated_at': '2016-11-04 13:10:00',
  'location': {'lon': 16.963, 'lat': 51.0163},
  'id': 21,
  'title': 'Osiedle Słoneczne',
  'url': '/inwestycja/osiedle-sloneczne,21.html'},
 {'updated_at': '2016-11-04 13:10:00',
  'location': {'lon': 16.9684, 'lat': 51.0298},
  'id': 22,
  'title': 'Osiedle Kameralne',
  'url': '/inwestycja/osiedle-kameralne,22.html'},
 {'updated_at': '2016-11-04 13:10:00',
  'location': {'lon': 16.9654, 'lat': 51.0345},
  'id': 23,
  'title': 'Ogrody Bielańskie',
  'url': '/inwestycja/ogrody-bielanskie,23.html'},
 {'updated_at': '2016-11-04 13:10:00',
  'location': {'lon': 16.972, 'lat': 51.0397},
  'id': 24,
  'title': 'Osiedle Filmowe (24 domki)',
  'url': '/inwestycja/osiedle-filmowe-24-domki,24.html'},
 {'updated_at': '2016-11-04 13:10:00',
  'location': {'lon': 17.3829, 'lat': 51.0461},

In [29]:
# set_handler()
for estate_json in json_response:
    print(estate_json)
    id = estate_json['id']
    updated_at = estate_json['updated_at']
    latitude = estate_json['location']['lat']
    longitude = estate_json['location']['lon']
    title = estate_json['title']
    url = check_text(estate_json['url'])

    insert_text = f"INSERT OR IGNORE INTO INVESTMAP VALUES ('{id}', '{updated_at}', {latitude}, {longitude}, '{title}', '{url}')"

    try:
        root.debug(insert_text)
        insert_result = cur.execute(insert_text, {'null':None})
        conn.commit()
    except Exception as error:
        root.error("Exception occurred", exc_info=True)
    break

{'updated_at': '2016-11-04 13:10:00', 'location': {'lon': 17.4609, 'lat': 50.8575}, 'id': 20, 'title': 'Liberty Park', 'url': '/inwestycja/liberty-park,20.html'}


In [116]:
    # Create table

try:
    root.debug(insert_text)
    insert_result = cur.execute(insert_text, {'null':None})
    conn.commit()
except Exception as error:
    root.error("Exception occurred", exc_info=True)

In [90]:
# id = 13 #check_if_null(json_response['id'])
# address = check_if_null(json_response['address'])
# construction_date_lower = check_if_null(json_response['construction_date_range']['lower'])
# construction_date_upper = check_if_null(json_response['construction_date_range']['upper'])
# coordinates = json_response['geo_point']['coordinates']
# name = check_if_null(json_response['name'])
# price_type = check_if_null(json_response['price_type'])
# region_full_name = check_if_null(json_response['region']['full_name'])
# region_id = check_if_null(json_response['region']['id'])
# region_short_name = check_if_null(json_response['region']['short_name_reverted'])
# slug = check_if_null(json_response['slug'])
# stats = json_response['stats'] #dict!
# street_name = check_if_null(json_response['street_name'])
# street_number = check_if_null(json_response['street_number'])
# type = check_if_null(json_response['type'])
# insert_text = f"INSERT OR IGNORE INTO RYNEK_PIERWOTNY VALUES ('{id}', '{address}', '{construction_date_lower}', '{construction_date_upper}', {coordinates[0]}, {coordinates[1]}, '{name}', {price_type}, '{region_full_name}', {region_id}, '{region_short_name}', {stats['distance_from_region']['center']}, '{stats['distance_from_region']['region']}', {stats['distance_from_region']['distance']}, {stats['properties_count_for_sale']}, {stats['properties_with_roi_count']}, {stats['ranges_area_max']}, {stats['ranges_area_min']}, {stats['ranges_price_m2_min']}, {stats['ranges_rooms_max']}, {stats['ranges_rooms_min']}, '{street_name}', {street_number}, {type})"
# insert_text

"INSERT OR IGNORE INTO RYNEK_PIERWOTNY VALUES ('13', 'Lublin, ul. Wolińskiego', '2020-12-31', '2023-03-31', 22.550818040092842, 51.20219089997789, 'ActiveCity II etap', 2, 'lubelskie, Lublin', 13220, 'Lublin', True, 'Lublina', 5.1, 20, 0, 79, 54, 6950, 4, 3, 'ul. Wolińskiego', :null, 1)"

In [91]:
# insert_result = cur.execute(insert_text, {'null':None})
# results = insert_result.fetchall()
# conn.commit()
# results

[]